# Write to MySQL Database From Google Sheets Using Python

Required Library to be install:

1. pip install gspread
2. pip install oauth2client
3. pip install mysql-connector
Now Follow these steps to access Google Sheet via python :
1. Go to the Google APIs Console.
2. Create a new Project.
3. Click Enable API. Search for and enable Google Drive API.
4. Create Credentials and Select WebServer and Application Data. In Project Role Select Editor.
5. Download JSON file save to your project folder and rename it as your required.

Open the JSON file and copy the cilent_email address and Share your Google Sheet with that Email ID with Write Permisssion. Otherwise you will get error SheetsNotFound

Note: When you run the script first time it gives error as to enable Google Sheet just click the link and its done.

In [1]:
import gspread
import MySQLCredentials as mc
import mysql.connector
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
scope = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]

In [3]:
creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scope)
client = gspread.authorize(creds)

In [4]:
# define method to pull data from spreadsheet
def GetSpreadsheetData(sheetName, worksheetIndex):    
    sheet = client.open(sheetName).get_worksheet(worksheetIndex)    
    return sheet.get_all_values()[1:]

In [11]:
# Client Email inside secret.json file must have edit permission on the Google Sheet you are using
horizontal_cct = GetSpreadsheetData('Horizontals CCT - 2021', 4)

In [12]:
# View Data From Google Sheets 
print(horizontal_cct)
print(len(horizontal_cct))

[['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '1', 'Electronics', 'Item 1', 'Calls', '1', '1'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '1', 'Electronics', 'Item 2', 'Calls', '1', '1'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '1', 'Vehicles', 'Item 1', 'Calls', '12', '12'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '1', 'Property Sales & Rentals', 'Item 1', 'Calls', '19', '19'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '1', 'Property Sales & Rentals', 'Item 2', 'Calls', '7', '7'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '1', 'Jobs', 'Item 1', 'Emails', '56', '56'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '2', 'Electronics', 'Item 1', 'Calls', '3', '3'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '2', 'Electronics', 'Item 2', 'Calls', '2', '2'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '2', 'Vehicles', 'Item 1', 'Calls', '7', '7'], ['2021-01-01', 'Senegal', 'Expat Dakar', 'Unpaid', '2', 'Property Sale

In [13]:
# define method to write list of data to MySQL table
def WriteToMySQLTable(sql_data, tableName):
# we are using a try/except block (also called a try/catch block in other languages) which is good for error handling. It will “try” to execute anything in the “try” block, and if there is an error, it will report the error in the “except” block. Regardless of any errors, the “finally” block will always be executed.
    try:
# Here we include the connection credentials for MySQL. We create a connection object that we pass the credentials to, and notice that we can specify the database which is ‘sys’ in the MySQLCredentials.py file because I’m using since I’m using the default database in MySQL Workbench 8.0.
        connection = mysql.connector.connect(
        user = mc.user,
        password = mc.password,
        host = mc.host,
        database = mc.database
        )
# This command will drop the table, and we could just have the table name hardcoded into the string, but instead I am using the name of the table passed into the method. {} is a placeholder for what we want to pass into this string, and using .format(blah) we can pass the string name from the variable passed into the method here.

         # Drop Group Table if it exists
        sql_drop = "DROP TABLE IF EXISTS {} ".format('cct_data')
        #create Group Table
        sql_create_table = """CREATE TABLE `cct_data` (
  `month` date NOT NULL,
  `country` varchar(300) COLLATE utf8mb4_unicode_ci NOT NULL,
  `platform` varchar(1000) COLLATE utf8mb4_unicode_ci NOT NULL,
  `type` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,
  `day` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,
  `category` varchar(1500) COLLATE utf8mb4_unicode_ci NOT NULL,
  `item` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,
  `kpi` varchar(500) COLLATE utf8mb4_unicode_ci NOT NULL,
  `value_1` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,
  `value_2` varchar(255) COLLATE utf8mb4_unicode_ci DEFAULT NULL,
  KEY `month` (`month`),
  KEY `country` (`country`),
  KEY `platform` (`platform`(768)),
  KEY `category` (`category`(768)),
  KEY `kpi` (`kpi`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci"""
    
    
    # Insert into cct_data Table From Google Sheet
        sql_insert_statement = """INSERT INTO {}(
        
           month, country, platform, type, day, category, item, kpi, value_1, value_2
           
           ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""".format('cct_data')
        
# Here we create a cursor, which we will use to execute the MySQL statements above. After each statement is executed, a message will be printed to the console if the execution was successful.
        cursor = connection.cursor()
        cursor.execute(sql_drop)
        print('Table {} has been dropped'.format('cct_data')) 
        cursor.execute(sql_create_table)        
        print('Table {} has been created'.format('cct_data'))
        """cursor.execute(sql_drop)
        print(‘Table {} has been dropped’.format(tableName))
        cursor.execute(sql_create_table)
        print(‘Table {} has been created’.format(tableName)) 
        """
# We need to write each row of data to the table, so we use a for loop that will insert each row of data one at a time
        for i in sql_data:
            cursor.execute(sql_insert_statement,i)
# Now we execute the commit statement, and print to the console that the table was updated successfully
        connection.commit()
        print("Table {} successfully updated.".format('cct_data'))
# Errors are handled in the except block, and we will get the information printed to the console if there is an error
    except mysql.connector.Error as error:        
        connection.rollback()        
        print("Error: {}. Table {} not updated!".format(error, 'cct_data'))
    # We need to close the cursor and the connection, and this needs to be done regardless of what happened above.
    finally:
        cursor.execute("SELECT COUNT(*) FROM {}".format('cct_data'))
        rowCount = cursor.fetchone()[0]
        print("cct_data", "row count:", rowCount)
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed.")

In [14]:
WriteToMySQLTable(horizontal_cct,'cct_data')

Table cct_data has been dropped
Table cct_data has been created
Table cct_data successfully updated.
cct_data row count: 6425
MySQL connection is closed.
